In [ ]:
#install pyspark
!pip3 install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 44 kB/s 
     |████████████████████████████████| 199 kB 46.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=cc909bb4e4f6601eaa2c1dec8685db43e406644c2e8e13031c0de2b03c011ad6
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [ ]:
#create a sparksession
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spark').getOrCreate()

In [ ]:
#clone the dataset
! git clone https://github.com/education454/admission_dataset

Cloning into 'admission_dataset'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (3/3), done.


In [ ]:
#checking the presence of dataset
! ls admission_dataset

Admission_Predict_Ver1.1.csv


In [ ]:
#creating a spark dataframe
df = spark.read.csv('/content/admission_dataset/Admission_Predict_Ver1.1.csv', header = True, inferSchema = True)

In [ ]:
#display dataframe
df.show()

+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|Serial No|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+---------+-----------+-----------------+---+---+----+--------+---------------+
|        1|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|        2|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|        3|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|        4|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|        5|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|        6|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|        7|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|        8|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|        9

In [ ]:
#getting the no.of rows & columns
print((df.count(), len(df.columns)))

(500, 9)


In [ ]:
#print schema 
df.printSchema()

root
 |-- Serial No: integer (nullable = true)
 |-- GRE Score: integer (nullable = true)
 |-- TOEFL Score: integer (nullable = true)
 |-- University Rating: integer (nullable = true)
 |-- SOP: double (nullable = true)
 |-- LOR: double (nullable = true)
 |-- CGPA: double (nullable = true)
 |-- Research: integer (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#getting the summary statistics
df.describe().show()

+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|summary|        Serial No|         GRE Score|      TOEFL Score|University Rating|               SOP|               LOR|              CGPA|          Research|    Chance of Admit|
+-------+-----------------+------------------+-----------------+-----------------+------------------+------------------+------------------+------------------+-------------------+
|  count|              500|               500|              500|              500|               500|               500|               500|               500|                500|
|   mean|            250.5|           316.472|          107.192|            3.114|             3.374|             3.484| 8.576440000000003|              0.56| 0.7217399999999996|
| stddev|144.4818327679989|11.295148372354712|6.081867659564538|1.143511800759815|0.9910036207566072|0.92

####Data Cleaning

In [ ]:
#droping the unnecessary column
df = df.drop('Serial No')

In [ ]:
#displaying the dataframe
df.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|
+---------+-----------+-----------------+---+---+----+--------+---------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75|
|      308|        101|                2|3.0|4.0| 7.9|       0|           0.68|
|      302|        102|                1|2.0|1.5| 8.0|       0|            0.5|
|      323|        108|                3

In [ ]:
#check for null values
for i in df.columns:
  print(i+":", df[df[i].isNull()].count())

GRE Score: 0
TOEFL Score: 0
University Rating: 0
SOP: 0
LOR: 0
CGPA: 0
Research: 0
Chance of Admit: 0


####Correlation Analysis & Feature Selection

In [ ]:
#correlation analysis
for col in df.columns:
  print('Correlation to chance of admit for {} is {}'.format(col, df.stat.corr('Chance of Admit', col)))

Correlation to chance of admit for GRE Score is 0.8103506354632598
Correlation to chance of admit for TOEFL Score is 0.7922276143050823
Correlation to chance of admit for University Rating is 0.6901323687886892
Correlation to chance of admit for SOP is 0.6841365241316723
Correlation to chance of admit for LOR is 0.6453645135280112
Correlation to chance of admit for CGPA is 0.882412574904574
Correlation to chance of admit for Research is 0.5458710294711379
Correlation to chance of admit for Chance of Admit is 1.0


In [ ]:
#feature selection
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['GRE Score', 'TOEFL Score', 'CGPA'], outputCol='features')

In [ ]:
#display dataframe
output_data = assembler.transform(df)
output_data.show()

+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|GRE Score|TOEFL Score|University Rating|SOP|LOR|CGPA|Research|Chance of Admit|          features|
+---------+-----------+-----------------+---+---+----+--------+---------------+------------------+
|      337|        118|                4|4.5|4.5|9.65|       1|           0.92|[337.0,118.0,9.65]|
|      324|        107|                4|4.0|4.5|8.87|       1|           0.76|[324.0,107.0,8.87]|
|      316|        104|                3|3.0|3.5| 8.0|       1|           0.72| [316.0,104.0,8.0]|
|      322|        110|                3|3.5|2.5|8.67|       1|            0.8|[322.0,110.0,8.67]|
|      314|        103|                2|2.0|3.0|8.21|       0|           0.65|[314.0,103.0,8.21]|
|      330|        115|                5|4.5|3.0|9.34|       1|            0.9|[330.0,115.0,9.34]|
|      321|        109|                3|3.0|4.0| 8.2|       1|           0.75| [321.0,109.0,8.2]|
|      308

Building the Linear Regression Model

In [ ]:
#import Linearregression and create final data
from pyspark.ml.regression import LinearRegression
final_data = output_data.select('features', 'Chance of Admit')

In [ ]:
#print schema of final data
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- Chance of Admit: double (nullable = true)



In [ ]:
#split the dataset into training and testing set
train, test = final_data.randomSplit([0.7,0.3])

In [ ]:
#build & train the model
models = LinearRegression(featuresCol='features', labelCol='Chance of Admit')
model = models.fit(train)

In [ ]:
#coefficients & intercept
print('Coefficients:', model.coefficients)
print('Intercept:', model.intercept)

Coefficients: [0.0025300220949539204,0.0034203163899985518,0.13811162294161844]
Intercept: -1.631431507680261


In [ ]:
#summary of the model
summary = model.summary


In [ ]:
#print the rmse & r2 score
print('RMSE:', summary.rootMeanSquaredError)
print('r2 score:', summary.r2)

RMSE: 0.06648881341019927
r2: 0.7765366948055861



Evaluate & Save the Model

In [ ]:
#transform on the test data
predictions = model.transform(test)

In [ ]:
#display the predictions
predictions.show(20)

+------------------+---------------+------------------+
|          features|Chance of Admit|        prediction|
+------------------+---------------+------------------+
|[290.0,100.0,7.56]|           0.47|0.4884304082948665|
| [294.0,93.0,7.36]|           0.46|0.4469859573563686|
| [294.0,95.0,7.64]|           0.49| 0.492497844560019|
|  [295.0,93.0,7.2]|           0.46|0.4274181197806639|
| [295.0,99.0,7.65]|           0.57|0.5100902484443832|
| [297.0,98.0,7.67]|           0.59|0.5144922087031247|
| [297.0,99.0,7.81]|           0.54|0.5372481523049499|
|[297.0,101.0,7.67]|           0.57|0.5247531578731204|
| [298.0,97.0,7.21]|           0.45|0.4500705678549357|
|[298.0,101.0,7.69]|           0.53|0.5300454124269067|
| [299.0,106.0,8.4]|           0.64|0.6477362687604025|
|  [300.0,99.0,6.8]|           0.36|0.4053454794187772|
|[300.0,101.0,7.88]|           0.59|0.5613466649757224|
| [300.0,105.0,7.8]|           0.58|0.5639790007003871|
| [301.0,96.0,7.56]|           0.54|0.5025793857

In [ ]:
#evaluate the model 
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(predictionCol='prediction', labelCol='Chance of Admit', metricName='r2')
print('r2 on the test data:', evaluator.evaluate(predictions))

r2 on the test data: 0.876740840324901


In [ ]:
#save the model
model.save("Admission Prediction Model")